# START HERE

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten,\
BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import itertools
import os
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
from skimage.transform import resize
from os import listdir, walk
from os.path import isfile, join
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K

import splitfolders

In [2]:
def fetch_files_names(directory):
    subfolders_path = [ f.name for f in os.scandir(directory) if f.is_dir() ]
    return subfolders_path

In [3]:
# directory of the base folder
dir = 'data/clean_data/'
# Create a dictionary with mashrooms and labels
CLASSES = dict(zip(fetch_files_names(dir), np.arange(0, len(fetch_files_names(dir)))))

In [5]:
# give the directory that includes folders of images and return three files split in train test validation

def split_data(dir = dir, output = dir + "/../output"):
    """
    Split data in train, test, validataion
    """
    splitfolders.ratio(dir , output=output, seed=1337, ratio=(.75, 0.15,0.1)) 


In [6]:
split_data()
train_path = dir + "/../output" + '/train/'
test_path = dir + "/../output" + '/test/'
valid_path = dir + "/../output" + '/val/'

Copying files: 26817 files [02:41, 166.13 files/s]


In [7]:
# prepare data for the model in batches to avoid RAM problems
train_datagen = ImageDataGenerator(rescale=1.0/255.0,validation_split=0.2) 
train_batches = train_datagen.flow_from_directory(directory=train_path, \
target_size=(224,224), classes=CLASSES, batch_size=64,subset='training')

validation_batches = train_datagen.flow_from_directory(directory=train_path, \
target_size=(224,224), classes=CLASSES, batch_size=64,subset='validation')

test_batches = ImageDataGenerator(rescale=1.0/255.0).flow_from_directory(\
directory=test_path, target_size=(224,224), classes=CLASSES, \
batch_size=64, shuffle=False)

Found 16094 images belonging to 26 classes.
Found 4009 images belonging to 26 classes.
Found 2705 images belonging to 26 classes.


# MODEL ResNet50V2

In [8]:
from tensorflow.keras.applications.resnet_v2 import ResNet50V2, decode_predictions, preprocess_input
from tensorflow.keras.preprocessing import image # Keras own inbuild image class
import os
from tensorflow.keras.models import Model
from tensorflow import keras

def model_ResNet50V2( learning_rate=0.001, cl = CLASSES):
    """
    This function returns a ResNet50V2 model with the last
    layer removed.
    """
    K.clear_session() # Always clear the session!

    base_model = ResNet50V2(
        weights='imagenet', 
        pooling='avg',      # applies global average pooling to the output of the last conv layer (like a flattening)
        include_top=False,   # !!!!! we only want to have the base, not the final dense layers 
        input_shape=(224, 224, 3)  
    )
    base_model.trainable = False # To freeze the model
    # Start building on top of the model
    model = keras.Sequential() # defining a new model
    model.add(base_model) # adding in the pretrained model without the fully connected layer
    model.add(keras.layers.Dense(64, activation='relu')) # adding in additional layers
    model.add(keras.layers.Dropout(0.2))
    model.add(keras.layers.Dense(128, activation='relu')) # adding in additional layers
    model.add(keras.layers.Dropout(0.2))
    model.add(keras.layers.Dense(len(cl), activation='softmax')) #!!! Final layer with a length of 2, and softmax activation 
    # have a look at the trainable and non-trainable params statistic
    model.summary()
    model.compile(optimizer=keras.optimizers.Adam(learning_rate),
              loss=keras.losses.categorical_crossentropy,
              metrics=[keras.metrics.categorical_accuracy])

    return model

def fit_ResNet50V2(train_batches=train_batches,validation_data =validation_batches,epochs = 200, sav = True):
    """
    This function fits the model on the training data.
    """
    # observe the validation loss and stop when it does not improve after 3 iterations
    callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

    history = model.fit( x=train_batches,
            epochs=epochs, 
            verbose=1, 
            callbacks=[callback],
            validation_data = validation_data, )
    
    # save model
    if sav == True:
        model.save('models/ResNet50V2_test.h5') 
    return model, history


In [ ]:
model = model_ResNet50V2(learning_rate=0.001)
model, history = fit_ResNet50V2(epochs = 200)

# Model InceptionResNetV2

In [ ]:
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2, decode_predictions, preprocess_input
from tensorflow.keras.preprocessing import image # Keras own inbuild image class
import os
from tensorflow.keras.models import Model
from tensorflow import keras

def model_InceptionResNetV2( learning_rate=0.001, cl = CLASSES):
    """
    This function returns a ResNet50V2 model with the last
    layer removed.
    """
    K.clear_session() # Always clear the session!

    base_model = InceptionResNetV2(
        weights='imagenet', 
        pooling='avg',      # applies global average pooling to the output of the last conv layer (like a flattening)
        include_top=False,   # !!!!! we only want to have the base, not the final dense layers 
        input_shape=(224, 224, 3)  
    )
    base_model.trainable = False # To freeze the model
    # Start building on top of the model
    model = keras.Sequential() # defining a new model
    model.add(base_model) # adding in the pretrained model without the fully connected layer
    model.add(keras.layers.Dense(64, activation='relu')) # adding in additional layers
    model.add(keras.layers.Dropout(0.2))
    model.add(keras.layers.Dense(128, activation='relu')) # adding in additional layers
    model.add(keras.layers.Dropout(0.2))
    model.add(keras.layers.Dense(len(cl), activation='softmax')) #!!! Final layer with a length of 2, and softmax activation 
    # have a look at the trainable and non-trainable params statistic
    model.summary()
    model.compile(optimizer=keras.optimizers.Adam(learning_rate),
              loss=keras.losses.categorical_crossentropy,
              metrics=[keras.metrics.categorical_accuracy])

    return model

def fit_ResNet50V2(train_batches=train_batches,validation_data =validation_batches,epochs = 200, sav = True):
    """
    This function fits the model on the training data.
    """
    # observe the validation loss and stop when it does not improve after 3 iterations
    callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

    history = model.fit( x=train_batches,
            epochs=epochs, 
            verbose=1, 
            callbacks=[callback],
            validation_data = validation_data, )
    
    # save model
    if sav == True:
        model.save('models/ResNet50V2_test.h5') 
    return model, history
